# Movie Cast Named Entity Recognition Using ChatGPT

The movie dataset, found on [kaggle.com](https://www.kaggle.com/datasets/utkarshx27/movies-dataset), contains a `cast` column that displays a string of all cast members of the movie.

An example `cast` entry is:
```
Robert Downey Jr. Chris Hemsworth Mark Ruffalo Chris Evans Scarlett Johansson
```

Other than this cast representing a Marvel movie, one obvious characteristic is that the space-separated name segments are without an indicator showing where a name starts and ends. As a result, a "simple" program that aims to parse the cast to analyze relationships between a cast member and other movie data would be challenging. The edge cases needed would be enormous resulting in the [If Statement Code Smell](https://blog.jetbrains.com/idea/2017/09/code-smells-if-statements/).

This is a great opportunity for Natural Language Processing (NLP) via OpenAI's ChatGPT to identify complete names in a `cast` string. The names should be comma-separated with a comma representing the end of one name and the beginning of another. An example use case of this format includes generating a dictionary of unique cast members.

## Prerequisites

1. Run `pip install -r requirements.txt`
2. Create a `.env` file in the root directory storing your `OPENAI_API_KEY`

In [25]:
import os

from dotenv import load_dotenv
import openai
import pandas as pd

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

## Processing a Single String

Continuing the intro's example, let's use an `openai` completion to find the names.

The target output is a comma-separated string of names that should look like this:
```
Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Chris Evans,Scarlett Johansson
```

In [26]:
def extract_names(text):
    """Converts a space-separated string of names into a comma-separated string of names.

    The method uses openai's completion API to parse the cast text and identify the complete
      names.

    Returns:
      A comma-separated string of names.
    """
    response = openai.Completion.create(
      engine="text-davinci-003",
      prompt=f'Detect the names in the string below, and return the complete names separated by commas:\n\n"{text}"',
      max_tokens=1000
    )

    # Store the output as a list of complete names.
    names = response.choices[0].text.strip().split(", ")

    # Join the complete names into a single string separated by commas.
    return ",".join(names)

cast_csv = extract_names("Robert Downey Jr. Chris Hemsworth Mark Ruffalo Chris Evans Scarlett Johansson")
print(cast_csv)

Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Chris Evans,Scarlett Johansson


## Update the Dataset's `cast` Value

Now that the problem with the current `cast` representation has been identified and a solution has been created, create a new column titled `cast_csv` that will contain the updated representation. 

In [28]:
df = pd.read_csv('./data/movie_dataset.csv')

# Limit the dataframe to the first 25 rows for example purposes.
df = df.head(25)

df['cast_csv'] = df['cast'].apply(extract_names)
df.to_csv('./data/movie_dataset_with_cast_csv_top_25.csv', index=False)

## Conclusion

The cast member complete names can now be found by splitting commas on the `cast_csv` column. This removes the guesswork of trying to determine a complete name using the space-separated `cast` column.

Example entries:
<br>
| cast | cast_csv |
| --- | --- |
| Robert Downey Jr. Chris Hemsworth Mark Ruffalo Chris Evans Scarlett Johansson  | Robert Downey Jr.,Chris Hemsworth,Mark Ruffalo,Chris Evans,Scarlett Johansson |
| Russell Crowe Cate Blanchett Max von Sydow William Hurt Mark Strong  | Russell Crowe,Cate Blanchett,Max von Sydow,William Hurt,Mark Strong |

## 

## 